## Magentic-One
https://microsoft.github.io/autogen/dev/user-guide/agentchat-user-guide/magentic-one.html

## Load Azure Configuration

In [2]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client

In [3]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient, OpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)


In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console


assistant = AssistantAgent(
    "Assistant",
    model_client=az_model_client,
)


team = MagenticOneGroupChat([assistant], model_client=az_model_client)
await Console(team.run_stream(task="Provide a different proof for Fermat's Last Theorem"))


In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

surfer = MultimodalWebSurfer(
        "WebSurfer",
        model_client=az_model_client,
    )

team = MagenticOneGroupChat([surfer], model_client=az_model_client)
response = await Console(team.run_stream(task="Is there a Lakers game on January 5, 2025?"))

In [ ]:
# ANSI escape code for bold text
bold_start = "\033[1m"
bold_end = "\033[0m"

# ANSI escape code for red text
red_start = "\033[31m"
red_end = "\033[0m"

for messages in response.messages:
    source = messages.source
    print(f"{bold_start}{red_start}{source}{bold_end}{red_end}")
    print(messages.content)

In [11]:
from autogen_core import CancellationToken
from autogen_ext.agents.openai import OpenAIAssistantAgent
from autogen_agentchat.messages import TextMessage
from openai import AsyncAzureOpenAI

cancellation_token = CancellationToken()

# Create AzureOpenAI client
client = AsyncAzureOpenAI(azure_endpoint=azure_openai_endpoint, 
                          api_version=azure_openai_api_version, 
                          api_key=azure_openai_key)

#### Create a file for code interpreter ####
file=open("../Data/assistant/bigfootsightings.csv", "rb")
code_interpreter_file = await client.files.create(
  file=file,
  purpose='assistants'
)
file.close()

#### Create an assistant with code interpreter and file search ####
assistant = OpenAIAssistantAgent(
    name="DataAssistant",
    description="Answer questions about data",
    client=client,
    model=azure_openai_deployment,
    instructions="You will use Python to answer questions about a dataset",
    tools=["code_interpreter"],
    tool_resources={"code_interpreter":{"file_ids":[code_interpreter_file.id]}}
)

team = MagenticOneGroupChat([assistant], model_client=az_model_client)
task = """
What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart with the sightings in the Y-axis and years in the X-axis and save it to a PNG file and sort by descending order.
Put the value on top of each bar
"""
response = await Console(team.run_stream(task=task))

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:412: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["vision"] is False:
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:427: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["json_output"] is False and json_output is True:
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:433: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["function_calling"] is False and len(tools) > 0:
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:419: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["json_output"] is False and json_output is True:


---------- user ----------

What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart with the sightings in the Y-axis and years in the X-axis and save it to a PNG file and sort by descending order.
Put the value on top of each bar

---------- MagenticOneOrchestrator ----------

We are working to address the following user request:


What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart with the sightings in the Y-axis and years in the X-axis and save it to a PNG file and sort by descending order.
Put the value on top of each bar



To answer this request we have assembled the following team:

DataAssistant: Answer questions about data


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
   - The request is about identifying the top 10 years with the most sightings.
   - There is mention of a "date column," which implies that there is a dataset

In [10]:
# Clean up resources
await assistant.delete_uploaded_files(cancellation_token)
await assistant.delete_assistant(cancellation_token)